In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import time
from keras.models import load_model
import pandas as pd
import sys
sys.path.append("D:\WindowsNoEditor\PythonAPI\carla")  # Adjust this path
import carla
import pygame
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent


pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)
# optional to load different towns
client.set_timeout(15)
client.load_world('Town06')
#define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('nissan')
#spawn a car in a random location

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

# move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)
carla_map = world.get_map()


In [3]:

class BatteryManager:
    def __init__(self, vehicle, destination, world, power_model, C_bat, P_aux):
        self.vehicle = vehicle
        self.destination = destination  # Renamed from point_b
        self.world = world
        self.power_model = power_model
        self.C_bat = C_bat
        self.P_aux = P_aux
        self.last_update_time = self.get_simulation_time()
        self.last_vehicle_location = self.vehicle.get_location()
        self.remaining_distance = self.compute_remaining_distance()
        self.soc = 100  # State of Charge in percentage
        self.total_energy = 0  # Total energy consumed (kWh)
        self.prev_velocity = self.vehicle.get_velocity()  # Initialize in constructor

    
    def get_simulation_time(self):
        return self.world.get_snapshot().timestamp.elapsed_seconds

    def compute_remaining_distance(self):
        carla_map = self.world.get_map()
        grp = GlobalRoutePlanner(carla_map, 2.0)
        route = grp.trace_route(self.vehicle.get_location(), self.destination)
        return sum(route[i][0].transform.location.distance(route[i + 1][0].transform.location) for i in range(len(route) - 1))

    def update_remaining_distance(self):
        current_time = self.get_simulation_time()
        vehicle_location = self.vehicle.get_location()
        
        self.remaining_distance = self.compute_remaining_distance()
        self.last_update_time = current_time
        self.last_vehicle_location = vehicle_location

    def get_slope(self):
        rotation = self.vehicle.get_transform().rotation
        return math.tan(math.radians(rotation.pitch))

    def compute_forces(self, speed_m_s, acceleration):
        slope = self.get_slope()
        speed_vector = np.array([speed_m_s, avg_speed])
        acc_vector = np.array([acceleration, avg_acc])
        slope_vector = np.array([slope, avg_slop])
        
        F_roll = c_rr * m * g * speed_vector / 1000
        F_climb = m * g * np.sin(slope_vector) * speed_vector / 1000
        F_aero = 0.5 * c_d * A * rho * speed_vector**2 * speed_vector / 1000
        F_acc = m * acc_vector * speed_vector  / 1000
        
        return F_roll, F_climb, F_aero, F_acc


    def get_acceleration(self,delta_time):
       
        
        if delta_time <= 0:
            return 0  # Avoid division by zero or negative time steps

        velocity = self.vehicle.get_velocity()
        speed_m_s = math.sqrt(velocity.x**2 + velocity.y**2)

        if hasattr(self, "prev_velocity") and self.prev_velocity is not None:
            prev_speed_m_s = math.sqrt(self.prev_velocity.x**2 + self.prev_velocity.y**2)
            acceleration = (speed_m_s - prev_speed_m_s) / delta_time
        else:
            acceleration = 0  # First time, assume no acceleration

        # Update for next calculation
        self.prev_velocity = velocity
        
        

        return acceleration


    


    def update_battery(self, delta_time):
        location = self.vehicle.get_location()
        velocity = self.vehicle.get_velocity()
        speed_m_s = np.sqrt(velocity.x**2 + velocity.y**2)
        acceleration = self.get_acceleration(delta_time)
        slope = self.get_slope() * 57.2958
        F_roll, F_climb, F_aero, F_acc = self.compute_forces(speed_m_s, acceleration)
        df = pd.DataFrame({
            'aeroF_P': F_aero,
            'climbF_P': F_climb,
            'rollF_P': F_roll,
            'accF_P': F_acc
        })
        P_net, req_P = -power_model.predict(df, verbose=0).flatten() #in kW
        P_net = P_net  * 0 if speed_m_s <=0 else P_net
        P_total = delta_time * (P_aux + P_net)  # Convert to kWh
        T_req_P = (P_aux + req_P)

        # Compute remaining distance
        #remaining_distance = abs(point_b.x - location.x)
        self.update_remaining_distance()
        remaining_distance = battery_manager.remaining_distance

        # Compute energy consumption
        energy_consumed = (1 / 3600) * P_total  # Convert W to kWh
        energy_required = (remaining_distance / avg_speed) / 3600 * T_req_P
        soc_required = (energy_required / C_bat) * 100
    
        
    
        self.total_energy += energy_consumed
        self.soc -= (energy_consumed / C_bat) * 100
        self.soc = min(max(self.soc, 0), 100)
        self.total_energy = max(0,self.total_energy)
        ETA = remaining_distance / (avg_speed + 0.01)
        return self.soc, soc_required, self.total_energy, ETA , remaining_distance,acceleration,slope


In [4]:


# Vehicle constants
m = 1680  # kg, Mass
g = 9.81  # m/s^2, Gravity
c_rr = 0.01  # Rolling resistance coefficient
c_d = 0.28  # Drag coefficient
A = 1.93  # m^2, Frontal area
rho = 1.20  # kg/m^3, Air density
C_bat = 24  # kWh, Battery capacity
alpha = 0.90  # Transmission efficiency
beta = 0.75  # Regeneration efficiency
P_aux = 0  # kW, Auxiliary power consumption
max_speed = 70 / 3.6  # m/s
avg_speed = 35 / 3.6
avg_slop = 0.002
avg_acc = 0.2

# Pygame initialization
pygame.init()
WIDTH, HEIGHT = 1600, 900  # Window size
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("CARLA Vehicle Info Display")
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Connect to CARLA
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)
length = 500
spawn_points = world.get_map().get_spawn_points()
destination = carla.Location(
    x=start_point.location.x - length,
    y=start_point.location.y,
    z=start_point.location.z,
)

# Load power model
power_model = load_model("c:\\Users\\HOUALEF\\Desktop\\RL_Carla\\models\\energy_model_best.hdf5")

# Initialize battery manager
battery_manager = BatteryManager(vehicle, destination, world, power_model,C_bat,P_aux)

# Attach a camera to the vehicle
camera_bp = world.get_blueprint_library().find("sensor.camera.rgb")
camera_bp.set_attribute("image_size_x", str(WIDTH))
camera_bp.set_attribute("image_size_y", str(HEIGHT))
camera_bp.set_attribute("fov", "110")
camera_transform = carla.Transform(
    carla.Location(x=-6.0, z=2.0),  # 6m behind, 2m above the vehicle
    carla.Rotation(pitch=-15)       # Optional: slight downward tilt
)
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
latest_image = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)

def process_img(image):
    global latest_image
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = np.reshape(array, (image.height, image.width, 4))
    latest_image = cv2.flip(array[:, :, :3], 1)

camera.listen(lambda image: process_img(image))

# Agent for automatic driving
agent = BehaviorAgent(vehicle, behavior="normal")
agent.set_destination(destination)

manual_mode = True
control = carla.VehicleControl()

# Main loop
running = True
while running:
    world.tick()
    clock.tick(20)
    
    current_time = battery_manager.get_simulation_time()
    delta_time = current_time - battery_manager.last_update_time
    battery_manager.last_update_time = current_time
    
    soc, soc_required, total_energy, ETA,remaining_distance,acceleration,slope = battery_manager.update_battery(
        delta_time
    )
    
    # Manual control
    keys = pygame.key.get_pressed()
    if keys[pygame.K_m]:
        manual_mode = not manual_mode
        time.sleep(0.2)
    
    if manual_mode:
        control.throttle = 1.0 if keys[pygame.K_UP] else 0.0
        control.brake = 1.0 if keys[pygame.K_DOWN] else 0.0
        control.steer = -0.5 if keys[pygame.K_LEFT] else (0.5 if keys[pygame.K_RIGHT] else 0.0)
        control.reverse = keys[pygame.K_SPACE]
    else:
        control = agent.run_step()
    
    vehicle.apply_control(control)
    
    # Pygame Display Update
    screen.fill((0, 0, 0))
    frame = cv2.cvtColor(latest_image, cv2.COLOR_BGR2RGB)
    frame = np.rot90(frame)
    frame = pygame.surfarray.make_surface(frame)
    screen.blit(frame, (0, 0))
    
    text_data = [
        f"Driving Mode: {'Manual' if manual_mode else 'Automatic'}",
        f"Speed: {vehicle.get_velocity().length() * 3.6:.2f} km/h",
        f"SoC: {soc:.2f}%",
        f"Energy Consumed: {total_energy:.4f} kWh",
        f"ETA: {ETA:.2f}s",
        f"Required SoC: {soc_required:.2f}%",
        f"Remaining Distance: {remaining_distance:.2f}m",
        f"Delta Time: {delta_time:.2f} s" ,
        f"Acceleration: {acceleration:.2f} m/s²",
        f"Slope: {slope:.2f} °",
    ]
    
    y_offset = 30
    for text in text_data:
        screen.blit(font.render(text, True, (255, 255, 255)), (20, y_offset))
        y_offset += 30
    
    pygame.display.flip()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT or soc <= 0:
            running = False

pygame.quit()
camera.destroy()
print("Simulation ended.")


Simulation ended.
